### Importo Librerias

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

### Cargo el Dataset

In [2]:
def load_data(df):
    texts = list(df['texto'].str.lower())
    labels = df['sentimiento'].tolist()
    return texts, labels

In [3]:
data_file = "./data.csv"
data = pd.read_csv(data_file)


In [4]:
data = data.dropna(subset=['texto'])

In [5]:
tf_count = data.value_counts("sentimiento")
tf_count

sentimiento
Negativo    5795
Positivo    5661
Neutral     5553
Name: count, dtype: int64

In [6]:
data = data[data['sentimiento'] != "Neutral"]

In [7]:
data['sentimiento'] = data['sentimiento'].replace({'Positivo':1, 'Negativo':0})

In [8]:
tf_count = data.value_counts("sentimiento")
tf_count

sentimiento
0    5795
1    5661
Name: count, dtype: int64

In [9]:
texts, labels = load_data(data)

In [10]:
class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
          self.texts = texts
          self.labels = labels
          self.tokenizer = tokenizer
          self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}


### Defino Modelo

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            x = self.dropout(pooled_output)
            logits = self.fc(x)
            return logits

In [12]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [13]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions,digits=4)


In [14]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        _, preds = torch.max(outputs, dim=1)
        print(torch.max(outputs, dim=1))
        return "Positive" if preds.item() == 1 else "Negative"

### Parametros


In [27]:
# Set up parameters
bert_model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 5
learning_rate = 3e-5

### Divido los Dataset en Train, Valid y Test

In [28]:
train_texts, test_val_texts, train_labels, test_val_labels = train_test_split(texts, labels, test_size=0.4, random_state=42)
test_texts, val_texts, test_labels, val_labels = train_test_split(test_val_texts, test_val_labels, test_size=0.5, random_state=42)

# Imprime el tamaño de cada conjunto
print("Tamaño del conjunto de entrenamiento:", len(train_texts), len(train_labels) )
print("Tamaño del conjunto de prueba:", len(test_texts), len(test_labels))
print("Tamaño del conjunto de validación:", len(val_texts), len(val_labels))

Tamaño del conjunto de entrenamiento: 6873 6873
Tamaño del conjunto de prueba: 2291 2291
Tamaño del conjunto de validación: 2292 2292


### Tokenizer

In [29]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [30]:
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

### Cargo Modelo

In [31]:
print(torch.cuda.is_available())

True


In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Entreno Modelo

In [34]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/5
Validation Accuracy: 0.8617
              precision    recall  f1-score   support

           0     0.8339    0.9018    0.8665      1141
           1     0.8941    0.8219    0.8565      1151

    accuracy                         0.8617      2292
   macro avg     0.8640    0.8619    0.8615      2292
weighted avg     0.8641    0.8617    0.8615      2292

Epoch 2/5
Validation Accuracy: 0.8647
              precision    recall  f1-score   support

           0     0.8788    0.8449    0.8615      1141
           1     0.8519    0.8844    0.8679      1151

    accuracy                         0.8647      2292
   macro avg     0.8653    0.8647    0.8647      2292
weighted avg     0.8653    0.8647    0.8647      2292

Epoch 3/5
Validation Accuracy: 0.8700
              precision    recall  f1-score   support

           0     0.8630    0.8782    0.8705      1141
           1     0.8771    0.8619    0.8694      1151

    accuracy                         0.8700      2292
   macro avg  

### Guardo el Modelo

In [40]:
torch.save(model.state_dict(), "bert_classifier.pth")


### Evaluo el modelo

In [36]:
accuracy, report = evaluate(model, test_dataloader, device)


In [37]:
print(f"Test Accuracy: {accuracy:.4f}")
print(report)

Test Accuracy: 0.8647
              precision    recall  f1-score   support

           0     0.8808    0.8540    0.8672      1185
           1     0.8485    0.8761    0.8621      1106

    accuracy                         0.8647      2291
   macro avg     0.8646    0.8651    0.8646      2291
weighted avg     0.8652    0.8647    0.8647      2291



In [38]:
test_text = "Que mal dia el de hoy."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(test_text)
print(f"Predicted sentiment: {sentiment}")

tensor([[ 4.0510, -3.9153]], device='cuda:0')
torch.return_types.max(
values=tensor([4.0510], device='cuda:0'),
indices=tensor([0], device='cuda:0'))
Que mal dia el de hoy.
Predicted sentimen: Negative
